In [1]:
from myhdl import block, delay, always_seq, instance, always, Signal, ResetSignal, traceSignals, now
from dataclasses import dataclass
from itertools import tee, product
from typing import Callable, Generator
from copy import deepcopy
from abc import ABC, abstractmethod, abstractproperty
import logging
from sys import version
logging.basicConfig(level=logging.INFO)
logging.info(version)

INFO:root:3.9.5 (default, Aug 29 2021, 19:01:31) 
[GCC 9.3.0]


In [2]:
@ block
def counter(clk, enable, reset, count):
    @always_seq(clk.posedge, reset=reset)
    def increment():
        if enable:
            count.next = count.val + 1
    return increment

@block
def clk_driver(clk, enable, period=20):
    lowTime = int(period / 2)
    highTime = period - lowTime

    @instance
    def drive_clk():
        while True:
            if not enable: 
                yield enable
            yield delay(lowTime)
            clk.next = 1
            yield delay(highTime)
            clk.next = 0

    return drive_clk

In [3]:
class AddressStreamDescriptor(ABC):
    def __init__(self):
        self._done = False
    
    @abstractmethod
    def __post_init__(self):
        pass 
    
    @abstractmethod
    def reset(self):
        pass
    
    @abstractmethod
    def __iter__(self):
        pass
    
    @abstractmethod
    def __next__(self):
        pass
    
    @property
    def done(self):
        return self._done
    
    @done.setter
    def done(self, val):
        if val:
            logging.debug("{} has concluded @T={}".format(self, now()))
        else:
            logging.debug("{} initialized @T={}".format(self, now()))
        self._done = val

@dataclass
class HighLevelAddressStreamDescriptor(AddressStreamDescriptor):
    index_generator_fn: Generator

    def __post_init__(self):
        super().__init__()
        self.done = False
        self.index_generator_fn, self.initial_index_generator_fn = tee(self.index_generator_fn)

    def reset(self):
        self.done = False
        self.index_generator_fn = self.initial_index_generator_fn
        self.index_generator_fn, self.initial_index_generator_fn = tee(self.index_generator_fn)

    def __iter__(self):
        return self

    def __next__(self):
        try:
            next_index = next(self.index_generator_fn)
        except StopIteration:
            next_index = 0
            self.done = True
        return next_index 

@dataclass
class LowLevelAddressStreamDescriptor(AddressStreamDescriptor):
    iteration_domain: Generator
    access_map: Callable
    condition: Callable

    def __post_init__(self):
        super().__init__()
        self.done = False
        self.initial_iteration_domain = deepcopy(self.iteration_domain)

    def reset(self):
        self.done = False
        self.iteration_domain = deepcopy(self.initial_iteration_domain)

    def __iter__(self):
        return self

    def __next__(self):
        try:
            next_iteration_vector = next(self.iteration_domain)
            print(next_iteration_vector)
            if self.condition(next_iteration_vector):
                next_index = self.access_map(next_iteration_vector)
            else:
                next_index = 0
        except StopIteration:
            next_index = 0
            self.done = True
        return next_index   


In [4]:
@ block
def stream_generator(clk, enable, reset, stream, global_counter, start_offset, stream_out):
    @always(clk.posedge, reset.posedge)
    def generate():
        if not reset and enable:
            if not stream.done:
                if global_counter >= start_offset:
                    next_idx = next(stream)
                    stream_out.next = next_idx
                    print("offset: {}, stream_out.next: {}".format(start_offset, next_idx))
        elif reset:
            stream.reset()
            stream_out.next = 0
    return generate

In [5]:
def chain_arch_pe_access_pattern(i_ub, j_ub, access_fn, i_lb = 0, i_step = 1, j_lb = 0, j_step = 1, condition = lambda i,j : True):
    for i in range(i_lb, i_ub, i_step):
        for j in range(j_lb, j_ub, j_step):
            if (condition(i, j)):
                yield access_fn(i, j)

def baseline_access_fn(pe_group , pe):
    return lambda i, j: i*10+j+1+pe_group*10+pe

@block
def top():
    clk = Signal(bool(0))
    enable = Signal(bool(0))
    global_counter = Signal(0)
    reset = ResetSignal(bool(0), active=1, isasync=True)
    counter_inst = counter(clk, enable, reset, global_counter)
    clk_driver_inst = clk_driver(clk, enable, period=10)

    stream_out_list = [Signal(0) for _ in range(9)]
    stream_descriptor_list = []
    for peGroup in range(3):
        for pe in range(3):
            stream_descriptor_list.append(HighLevelAddressStreamDescriptor(
                chain_arch_pe_access_pattern(8, 8, baseline_access_fn(peGroup, pe))))
            
    stream_generator_list = []
    for peGroup in range(3):
        for pe in range(3):
            pe_idx = peGroup*3 + pe
            stream_generator_list.append(stream_generator(
                clk, enable, reset, stream_descriptor_list[pe_idx], global_counter, pe_idx, stream_out_list[pe_idx]))

    @instance
    def start_sim():
        # reset cycle
        enable.next = 0
        reset.next = 1
        yield delay(10)
        enable.next = 1
        reset.next = 0

    return clk_driver_inst, counter_inst, start_sim, stream_generator_list


In [6]:
stream_descriptor_list = []
for peGroup in range(3):
    for pe in range(3):
        stream_descriptor_list.append(HighLevelAddressStreamDescriptor(
            chain_arch_pe_access_pattern(8, 8, lambda i, j: i*10+j+1+peGroup*10+pe)))

In [7]:
traceSignals.filename = 'Top'
traceSignals.tracebackup = False

In [8]:
dut = top()
inst = traceSignals(dut)
inst.run_sim(1200)
inst.quit_sim()

offset: 0, stream_out.next: 1
offset: 0, stream_out.next: 2
offset: 1, stream_out.next: 2
offset: 2, stream_out.next: 3
offset: 1, stream_out.next: 3
offset: 0, stream_out.next: 3
offset: 0, stream_out.next: 4
offset: 1, stream_out.next: 4
offset: 2, stream_out.next: 4
offset: 3, stream_out.next: 11
offset: 4, stream_out.next: 12
offset: 3, stream_out.next: 12
offset: 2, stream_out.next: 5
offset: 1, stream_out.next: 5
offset: 0, stream_out.next: 5
offset: 0, stream_out.next: 6
offset: 1, stream_out.next: 6
offset: 2, stream_out.next: 6
offset: 3, stream_out.next: 13
offset: 4, stream_out.next: 13
offset: 5, stream_out.next: 13
offset: 6, stream_out.next: 21
offset: 5, stream_out.next: 14
offset: 4, stream_out.next: 14
offset: 3, stream_out.next: 14
offset: 2, stream_out.next: 7
offset: 1, stream_out.next: 7
offset: 0, stream_out.next: 7
offset: 0, stream_out.next: 8
offset: 1, stream_out.next: 8
offset: 2, stream_out.next: 8
offset: 3, stream_out.next: 15
offset: 4, stream_out.next: 1

<class 'myhdl._SuspendSimulation'>: Simulated 1200 timesteps


In [9]:
peGroup = 1
pe = 0
for i in range(8):
    for j in range(8):
        print(i*10+j+1+peGroup*10+pe)

11
12
13
14
15
16
17
18
21
22
23
24
25
26
27
28
31
32
33
34
35
36
37
38
41
42
43
44
45
46
47
48
51
52
53
54
55
56
57
58
61
62
63
64
65
66
67
68
71
72
73
74
75
76
77
78
81
82
83
84
85
86
87
88


In [10]:
stream_descriptor_list = []
for peGroup in range(3):
    for pe in range(3):
        pe_idx = peGroup*3 + pe
        print(list(chain_arch_pe_access_pattern(8, 8, lambda i, j: i*10+j+1+peGroup*10+pe)))

[1, 2, 3, 4, 5, 6, 7, 8, 11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 48, 51, 52, 53, 54, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 71, 72, 73, 74, 75, 76, 77, 78]
[2, 3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15, 16, 17, 18, 19, 22, 23, 24, 25, 26, 27, 28, 29, 32, 33, 34, 35, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 48, 49, 52, 53, 54, 55, 56, 57, 58, 59, 62, 63, 64, 65, 66, 67, 68, 69, 72, 73, 74, 75, 76, 77, 78, 79]
[3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 33, 34, 35, 36, 37, 38, 39, 40, 43, 44, 45, 46, 47, 48, 49, 50, 53, 54, 55, 56, 57, 58, 59, 60, 63, 64, 65, 66, 67, 68, 69, 70, 73, 74, 75, 76, 77, 78, 79, 80]
[11, 12, 13, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25, 26, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 48, 51, 52, 53, 54, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 67, 68, 71, 72, 73, 74, 75, 76, 77, 78, 81, 82, 83, 84, 85, 86, 87,

In [11]:
t = 60

In [12]:
l = lambda x: x+t

In [13]:
l(10)

70